<a href="https://colab.research.google.com/github/dwahast/Deep-learning/blob/master/MLP_PyTorch_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons
- Link útil (https://towardsdatascience.com/cifar-10-image-classification-in-tensorflow-5b501f7dc77c)

In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F # softmax

#import pandas as pd # organize files 


In [2]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)
print(len(test_loader))

10000


In [0]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
        self.activation_function = nn.Sigmoid()
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x
      
class MLP_relu(nn.Module):
    def __init__(self):
        super(MLP_relu, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim =1)
        return x
      
      
class MLP_Srelu(nn.Module):
    def __init__(self):
        super(MLP_Srelu, self).__init__()
        self.fc1 = nn.Linear(32*32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 10)
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x),dim =1)
        return x   


In [5]:
model = MLP_Srelu()
print(model)

MLP_Srelu(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
)


In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

def opt_lss_lr_set(optimizer_method, loss_function, learning_rate):
   
  if loss_function == "mse":
    loss_fn = torch.nn.MSELoss()
  elif loss_function == "nllos":
     loss_fn = torch.nn.NLLoss()
  elif loss_function == "cross":
     loss_fn = torch.nn.CrossEntropyLoss()

  if optimizer_method == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.5)
  elif optimizer_method == "adadelta":
    optimizer = torch.optim.Adadelta(model.parameters(), lr = learning_rate)
  elif optimizer_method == "adagrad":
    optimizer = torch.optim.Adagrad(model.parameters(), lr = learning_rate)
  elif optimizer_method == "adamax":
    optimizer = torch.optim.Adamax(model.parameters(), lr = learning_rate)
  elif optimizer_method == "rms":
    optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)
  
  print("Opt Method:", optimizer_method.upper(), "| Loss Function:", loss_function.upper(), "| Learning Rate:", learning_rate)
  
  return optimizer, loss_fn

In [0]:
def one_hot(label, output_size):
    
    label_select = np.zeros(output_size)
    label_select[label]=1
    
    return torch.Tensor(label_select)

In [0]:
def evaluate(model, loss_fn, loss_function):
  model.eval() # set model to Evaluate "mode"
  losses = []
  corrects = 0
  current_total = 0
  accuracies = []
  
  for image, label in test_loader:
    weights = model(image)
   
    if(loss_function == 'mse'):
      label_target = one_hot(label,10) # one_hot to set the training class in the escalar
      loss = loss_fn(weights,label_target)
    else:
      loss = loss_fn(weights,label)
      
    losses.append(loss.item())
    _, predicted = torch.max(weights.data, 1)
    if(predicted==label):
      corrects += 1
    
    current_total += 1
    accuracies.append(corrects/current_total)
 
 # print("Mean Accuracy: ", np.mean(accuracies))
  return np.mean(accuracies), np.mean(losses)
      

In [0]:
# Realizar o treinamento aqui
def fit(epochs, optimizer_method, loss_function, learning_rate):
 
  optimizer, loss_fn = opt_lss_lr_set(optimizer_method, loss_function, learning_rate) #optimizer loss and Learning rate setter
  Accuracies = []
  Test_losses = []
  losses = []
  for epoch in range(epochs):
    model.train() # Set model to TRAIN "mode" (can be set to False for Test)
    epoch_losses = []
    for image,label in train_loader:
      optimizer.zero_grad()  # cleaning gradients between mini batches
      weights = model(image) 
     
      if(loss_function == 'mse'):
        label_target = one_hot(label,10) # one_hot to set the training class in the escalar
        loss = loss_fn(weights,label_target)
      else: 
        loss = loss_fn(weights, label)
        
      loss.backward() # Backpropagation 
      optimizer.step() # Optimization Method
      
      epoch_losses.append(loss.item())

    losses.append(np.mean(epoch_losses)) # Append mean losses for each epoch
   
    acc, test_loss = evaluate(model, loss_fn, loss_function)
    Accuracies.append(acc)
    Test_losses.append(test_loss)
    if(epoch%10==0):
      print("Epoch:",epoch, "- Average loss:", np.mean(epoch_losses),"- Accuracy:", acc) # Print mean Loss for each epoch
    
  
  print("Mean Accuracies:",np.mean(Accuracies))
  plt.title("Accuracies")
  plt.plot(Accuracies)
  plt.show()
  plt.title("Test Loss")
  plt.plot(Test_losses)
  plt.show()
 
  pass

In [10]:
# Avaliar o modelo aqui (no conjunto de teste)
fit(15,"sgd",    "mse", 0.01)
fit(15,"adadelta",   "mse", 0.01)
fit(15,"rms",    "mse", 0.01)
fir(15,"adagrad", "mse", 0.01)

KeyboardInterrupt: ignored